In [ ]:
import subprocess
import shlex
import json
import re

import numpy as np

from pathlib import Path
from IPython.display import display, Markdown

In [ ]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

In [ ]:
target_directory = json.loads(subprocess.run("cargo metadata --format-version 1".split(), capture_output=True).stdout)["target_directory"]

In [ ]:
def simulate_cpu_reporting_dynamic(
    algorithm, speciation=0.001, seed=42, sample=1.0, radius=564, sigma=10.0, report_speciation=False, report_dispersal=False, output="profile",
):
    if report_dispersal:
        reporters = ["Execution()", "Biodiversity()", "Counter()"]
    elif report_speciation:
        reporters = ["Execution()", "Biodiversity()"]
    else:
        reporters = ["Execution()"]
        
    config = "".join(f"""
    (
        speciation: {speciation},
        sample: {sample},
        seed: {seed},

        algorithm: {algorithm},

        scenario: AlmostInfinite(
            radius: {radius},
            sigma: {sigma},
        ),

        reporters: [
            Plugin(
                library: "{target_directory}/release/deps/libnecsim_plugins_common.so",
                reporters: [{', '.join(reporters)}]
            )
        ],
    )
    """.split()).replace(",)", ")").replace(",]", "]")
    
    # Profile the full simulation using perf, then filter the events and output for the Firefox Profiler
    print(
        f"perf record -F {997} --call-graph dwarf -g {target_directory}/release/rustcoalescence simulate '{config}'" +
        f" && perf script -F +pid > {output}.perf"
    )

In [ ]:
subprocess.run(shlex.split(
    f"cargo build --manifest-path {target_directory}/../rustcoalescence/Cargo.toml --release "
    + "--features rustcoalescence-algorithms-monolithic,rustcoalescence-algorithms-independent,"
    + "rustcoalescence-algorithms-cuda"
), check=True, capture_output=True, text=True);

In [ ]:
display(Markdown("# RAM Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("free"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

display(Markdown("# CPU Information:"))

display(Markdown('>```\n' + subprocess.run(
    shlex.split("lscpu"), check=True, capture_output=True, text=True
).stdout.replace('\n', '\n>') + '```'))

In [ ]:
seed = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64")

for algorithm in ["Classical()", "Gillespie()", "SkippingGillespie()", "Independent()"]:
    for report_speciation, report_dispersal, output in [
        (False, False, "progress"), (True, False, "progress-speciation"), (True, True, "progress-speciation-dispersal")
    ]:
        simulate_cpu_reporting_dynamic(
            algorithm, speciation=0.001, sample=1.0, radius=178, sigma=0.0, seed=seed,
            output=f"{CC_PATTERN.sub('-', algorithm[:algorithm.find('(')]).lower()}-{output}",
            report_speciation=report_speciation, report_dispersal=report_dispersal,
        )